# AutoGen

DOC [LiteLLM with Ollama](https://microsoft.github.io/autogen/0.2/docs/topics/non-openai-models/local-litellm-ollama)

!litellm --model ollama/llama3:latest

This will run the proxy server and it will be available at ‘http://0.0.0.0:4000/’.

In [1]:
import ollama
reply = ollama.list() # list all models
[model.model for model in reply.models]

['llama3:latest']

In [ ]:
# Single-agent example using Ollama LiteLLM
from autogen import ConversableAgent, UserProxyAgent
local_llm_config = {
    "config_list": [
        {
            "model": "NotRequired",  
            "api_key": "NotRequired", 
            "base_url": "http://localhost:4000",  
            "price": [0, 0],  
        }
    ],
    "cache_seed": None, 
}
assistant = ConversableAgent("agent", llm_config=local_llm_config)
user_proxy = UserProxyAgent("user", code_execution_config=False)
res = assistant.initiate_chat(user_proxy, message="How can I help you today?")
print(assistant)

agent (to user):

How can I help you today?

--------------------------------------------------------------------------------
user (to agent):

Tell me a joke

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent (to user):

Here's one:

Why couldn't the bicycle stand up by itself?

(wait for it...)

Because it was two-tired!

Hope that made you smile! Do you want another one?

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (ae4ffb6e-519d-4224-8b41-f1fe53714936): User requested to end the conversation
agent


## Tool Use with Gemini

DOC [Tool Use](https://microsoft.github.io/autogen/0.2/docs/tutorial/tool-use)

[Avaiable Models](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/models.html)

In [ ]:
# Creating Tools
import os
from typing import Annotated, Literal

Operator = Literal["+", "-", "*", "/"]

def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")
    
with open('gemini_key.txt', 'r') as f:
    gemini_key = f.read().strip()

gemini_llm_config = {
    "config_list": [
        {
            "model": "gemini-2.0-flash", 
            "api_key": gemini_key,  
            "api_type": "google",
        }
    ],
    "cache_seed": None,  
}

In [12]:
# Registering Tools
import warnings
warnings.filterwarnings("ignore")
from autogen import ConversableAgent

assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "After answering, reply with 'TERMINATE' to end the conversation.",
    llm_config=gemini_llm_config,
)

user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)
user_proxy.register_for_execution(name="calculator")(calculator)

from autogen import register_function
register_function(
    calculator,
    caller=assistant,  
    executor=user_proxy,  
    name="calculator",  
    description="A simple calculator",  
)
assistant.llm_config["tools"]

[{'type': 'function',
  'function': {'description': 'A simple calculator',
   'name': 'calculator',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer', 'description': 'a'},
     'b': {'type': 'integer', 'description': 'b'},
     'operator': {'enum': ['+', '-', '*', '/'],
      'type': 'string',
      'description': 'operator'}},
    'required': ['a', 'b', 'operator']}}}]

In [13]:
# Using Tools
chat_result = user_proxy.initiate_chat(assistant, message="4 + (5 * 3)?")

User (to Assistant):

4 + (5 * 3)?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):


***** Suggested tool call (3757): calculator *****
Arguments: 
{"b": 3, "a": 5, "operator": "*"}
**************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
Call ID: 3757
Input arguments: {'b': 3, 'a': 5, 'operator': '*'}

>>>>>>>> EXECUTED FUNCTION calculator...
Call ID: 3757
Input arguments: {'b': 3, 'a': 5, 'operator': '*'}
Output:
15
User (to Assistant):

***** Response from calling tool (3757) *****
15
*********************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):


***** Suggested tool call (1098): calculator *****
Arguments: 
{"a": 4, "b": 15, "operator": "+"}
************************

Structured output - Chain-of-Thought

DOC [Agents](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/agents.html)

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

from typing import Literal
from pydantic import BaseModel

class AgentResponse(BaseModel):
    thoughts: str
    response: Literal["happy", "sad", "neutral"]

model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=gemini_key, 
)

agent = AssistantAgent(
    "assistant",
    model_client=model_client,
    system_message="Categorize the input as happy, sad, or neutral following the JSON format.",
    output_content_type=AgentResponse,
)

result = await Console(agent.run_stream(task="I'm not feeling well."))

assert isinstance(result.messages[-1], StructuredMessage)
assert isinstance(result.messages[-1].content, AgentResponse)
print("Thought: ", result.messages[-1].content.thoughts)
print("Response: ", result.messages[-1].content.response)
await model_client.close()

---------- TextMessage (user) ----------
I'm not feeling well.
---------- StructuredMessage[AgentResponse] (assistant) ----------
{"thoughts":"The user reported feeling unwell, suggesting a negative emotional state.","response":"sad"}
Thought:  The user reported feeling unwell, suggesting a negative emotional state.
Response:  sad


In [26]:
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."

model_client = OpenAIChatCompletionClient(
    model="gemini-1.5-flash-8b",
    api_key=gemini_key, 
)

agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)

In [ ]:
result = await agent.run(task="Find information on AutoGen")
print(result.messages)
print(result.messages[-1].content)

[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 18, 15, 15, 10, 788622, tzinfo=datetime.timezone.utc), content='Find information on AutoGen', type='TextMessage'), ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=52, completion_tokens=6), metadata={}, created_at=datetime.datetime(2025, 6, 18, 15, 15, 11, 302451, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='', arguments='{"query":"AutoGen"}', name='web_search')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 18, 15, 15, 11, 302451, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', name='web_search', call_id='', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant', models_usage=None, metadata={}, created_at=datetime